<a href="https://colab.research.google.com/github/sumaaithal/PySpark_30Days_Challenge/blob/main/pyspark23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,582 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,611 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,332 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,047 kB]
Hit:13 https://ppa.launchp

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [25]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, percent_rank, cume_dist,lag, lead,col,avg,sum,min,max,row_number

In [4]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )

columns= ["employee_name", "department", "salary"]

In [6]:
df = spark.createDataFrame(data=simpleData,schema=columns)

In [7]:
df.printSchema()
df.show()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [9]:
windowSpec = Window.partitionBy("department").orderBy("salary")

In [10]:
df.withColumn("row_number",
              row_number().over(windowSpec) ).show(truncate=False)

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|Maria        |Finance   |3000  |1         |
|Scott        |Finance   |3300  |2         |
|Jen          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jeff         |Marketing |3000  |2         |
|James        |Sales     |3000  |1         |
|James        |Sales     |3000  |2         |
|Robert       |Sales     |4100  |3         |
|Saif         |Sales     |4100  |4         |
|Michael      |Sales     |4600  |5         |
+-------------+----------+------+----------+



In [14]:
df.withColumn("rank",
             rank().over(windowSpec)).show(truncate=False)

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|Maria        |Finance   |3000  |1   |
|Scott        |Finance   |3300  |2   |
|Jen          |Finance   |3900  |3   |
|Kumar        |Marketing |2000  |1   |
|Jeff         |Marketing |3000  |2   |
|James        |Sales     |3000  |1   |
|James        |Sales     |3000  |1   |
|Robert       |Sales     |4100  |3   |
|Saif         |Sales     |4100  |3   |
|Michael      |Sales     |4600  |5   |
+-------------+----------+------+----+



In [16]:
df.withColumn("dense_rank",
              dense_rank().over(windowSpec) ).show(truncate=False)

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|Maria        |Finance   |3000  |1         |
|Scott        |Finance   |3300  |2         |
|Jen          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jeff         |Marketing |3000  |2         |
|James        |Sales     |3000  |1         |
|James        |Sales     |3000  |1         |
|Robert       |Sales     |4100  |2         |
|Saif         |Sales     |4100  |2         |
|Michael      |Sales     |4600  |3         |
+-------------+----------+------+----------+



In [18]:
df.withColumn("percent_rank",
              percent_rank().over(windowSpec) ).show(truncate=False)

+-------------+----------+------+------------+
|employee_name|department|salary|percent_rank|
+-------------+----------+------+------------+
|Maria        |Finance   |3000  |0.0         |
|Scott        |Finance   |3300  |0.5         |
|Jen          |Finance   |3900  |1.0         |
|Kumar        |Marketing |2000  |0.0         |
|Jeff         |Marketing |3000  |1.0         |
|James        |Sales     |3000  |0.0         |
|James        |Sales     |3000  |0.0         |
|Robert       |Sales     |4100  |0.5         |
|Saif         |Sales     |4100  |0.5         |
|Michael      |Sales     |4600  |1.0         |
+-------------+----------+------+------------+



In [20]:
df.withColumn('cum_distribution',
               cume_dist().over(windowSpec) ).show(truncate=False)

+-------------+----------+------+------------------+
|employee_name|department|salary|cum_distribution  |
+-------------+----------+------+------------------+
|Maria        |Finance   |3000  |0.3333333333333333|
|Scott        |Finance   |3300  |0.6666666666666666|
|Jen          |Finance   |3900  |1.0               |
|Kumar        |Marketing |2000  |0.5               |
|Jeff         |Marketing |3000  |1.0               |
|James        |Sales     |3000  |0.4               |
|James        |Sales     |3000  |0.4               |
|Robert       |Sales     |4100  |0.8               |
|Saif         |Sales     |4100  |0.8               |
|Michael      |Sales     |4600  |1.0               |
+-------------+----------+------+------------------+



In [22]:
df.withColumn("lag",
             lag("salary",1).over(windowSpec) ).show(truncate=False)

+-------------+----------+------+----+
|employee_name|department|salary|lag |
+-------------+----------+------+----+
|Maria        |Finance   |3000  |NULL|
|Scott        |Finance   |3300  |3000|
|Jen          |Finance   |3900  |3300|
|Kumar        |Marketing |2000  |NULL|
|Jeff         |Marketing |3000  |2000|
|James        |Sales     |3000  |NULL|
|James        |Sales     |3000  |3000|
|Robert       |Sales     |4100  |3000|
|Saif         |Sales     |4100  |4100|
|Michael      |Sales     |4600  |4100|
+-------------+----------+------+----+



In [24]:
df.withColumn("lead",
             lead("salary",1).over(windowSpec) ).show(truncate=False)

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|Maria        |Finance   |3000  |3300|
|Scott        |Finance   |3300  |3900|
|Jen          |Finance   |3900  |NULL|
|Kumar        |Marketing |2000  |3000|
|Jeff         |Marketing |3000  |NULL|
|James        |Sales     |3000  |3000|
|James        |Sales     |3000  |4100|
|Robert       |Sales     |4100  |4100|
|Saif         |Sales     |4100  |4600|
|Michael      |Sales     |4600  |NULL|
+-------------+----------+------+----+



In [26]:
windowSpecAgg = Window.partitionBy("department")

In [34]:
df.withColumn("row_number",
              row_number().over(windowSpec))\
  .withColumn("avg",
              avg(col("salary")).over(windowSpecAgg))\
  .withColumn("min",
             min(col("salary")).over(windowSpecAgg))\
  .withColumn("sum",
             sum(col("salary")).over(windowSpecAgg))\
  .withColumn("max",
              max(col("salary")).over(windowSpecAgg))\
  .where(col("row_number")==1).select("department","avg","sum","min","max").show(truncate=False)

+----------+------+-----+----+----+
|department|avg   |sum  |min |max |
+----------+------+-----+----+----+
|Finance   |3400.0|10200|3000|3900|
|Marketing |2500.0|5000 |2000|3000|
|Sales     |3760.0|18800|3000|4600|
+----------+------+-----+----+----+

